<a href="https://colab.research.google.com/github/hhnafis/Project-3-LangChain-Function-Tool-Calling-Calculator/blob/main/langchain_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 3: Building a LangChain Calculator with Google Gemini Flash
## Langchain tool calling

In [1]:
#Install dependencies
!pip install langchain langchain-google-genai python-dotenv langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


In [2]:
#Loading API key
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_GEMINI_API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY")

In [21]:
#Defining calculator tool with optional enhancements
import math

class Calculator:
    def calculate(self, expression: str) -> str:
      try:
        if any(func in expression.lower() for func in ['log', 'sin', 'cos', 'tan', 'sqrt']):

          #Handle logrithmic problems
          if 'log' in expression.lower():
            try:
              base,num = map(float(expression.lower().replace('log', '').split('')))
              result = math.log(num, base)
              return str(result)
            except:
              try:
                num = float(expression.lower().replace('log','').split(''))
                result = math.log10(num)
                return str(result)
              except ValueError as e:
                return f"Error: {e}. Invalid logarithm input"

            #Handle trignometery problems
          elif 'sin' in expression.lower():
            angle = float(expression.lower().replace('sin','').split(''))
            result = math.sin(math.radians(angle))
            return str(result)
          elif 'cos' in expression.lower():
            angle = float(expression.lower().replace('cos','').split(''))
            result = math.cos(math.radians(angle))
            return str(result)
          elif 'tan' in expression.lower():
            angle = float(expression.lower().replace('tan','').split(''))
            result = math.tan(math.radians(angle))
            return str(result)
          elif 'sqrt' in expression.lower():
            num = float(expression.lower().replace('sqrt','').split(''))
            result = math.sqrt(num)
            return str(result)
          else:
            return "Invalid trigonometric function"
        else:

          #Handle simple arithmatic problems found in the expression
          result = eval(expression, {'__builtins__':None},{})
          return str(result)
      except Exception as e:
        return f"Error: {e}"




In [22]:
#Created the Tool Wrapper for LangChain
from langchain_core.tools import tool

@tool
def calculator(expression:str) -> str:
  """
  Perform mathematical calculations.
  Input: A mathematical expression as a string (e.g., "2 + 2")
  Output: Result of the calculation as a string. """

  calc = Calculator()
  return calc.calculate(expression)

In [23]:
#Seting Up the Google Gemini Flash Model
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GOOGLE_GEMINI_API_KEY= userdata.get('GOOGLE_API_KEY')

# Initializing the Gemini model
gemini_model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash",api_key=GOOGLE_GEMINI_API_KEY)


In [24]:
# Initializing LLM with tool-calling capabilities.

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents import AgentExecutor
tools = [calculator]

# Initializing the agent with tool-calling capabilities
agent = initialize_agent(
    tools, gemini_model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
!pip install langchain --upgrade #Upgraded langchain to the newest version
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# Seting up memory to maintain conversation context
memory = ConversationBufferMemory()

# Defining a prompt template that incorporates the conversation history
template = """You are a helpful assistant having a conversation with a user.
Current Conversation:
{history}
Human: {human_input}
Assistant:"""
prompt = PromptTemplate(
    input_variables=["history", "human_input"], template=template
)

# Build ingthe LLMChain
llm_chain = LLMChain(llm=gemini_model, prompt=prompt, memory=memory)

# Function to interact with the chain and agent
def conversational_agent(user_input):
  # Get LLM response considering conversation history
  llm_response = llm_chain.run(human_input=user_input)

  # If the response suggests using a tool, call the agent
  if "calculator" in llm_response.lower():
    agent_response = executor.invoke(llm_response)  # Use executor for tool calls
    return agent_response
  else:
    return llm_response



In [26]:
#Simple queries
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", conversational_agent(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?
Response: 25 multiplied by 4 is 100.
----------------------------------------
Query: Now divide the result by 5.
Response: 100 divided by 5 is 20.
----------------------------------------
Query: Add 10 to that.
Response: 20 plus 10 is 30.
----------------------------------------


In [29]:
#Optional enhancement

queries = [
    "Calculate the log of 100 to the base 10.",
    "What is the square root of 64?",
    "Find the tangent of 45 degrees.",
    "Find the cosine of 30 degrees."
]

for q in queries:
    print("Query:", q)
    print("Response:", conversational_agent(q))
    print("-" * 40)

Query: Calculate the log of 100 to the base 10.
Response: The logarithm base 10 of 100 is 2. This is because 10 raised to the power of 2 equals 100.
----------------------------------------
Query: What is the square root of 64?
Response: The square root of 64 is 8.
----------------------------------------
Query: Find the tangent of 45 degrees.
Response: The tangent of 45 degrees is 1.
----------------------------------------
Query: Find the cosine of 30 degrees.
Response: The cosine of 30 degrees is √3/2, or approximately 0.866.
----------------------------------------


In [30]:
print(conversational_agent("What are the calculations, I asked uptill here?"))

Here's a summary of the calculations you asked me to perform:

1. **25 multiplied by 4:** 25 * 4 = 100
2. **100 divided by 5:** 100 / 5 = 20
3. **20 plus 10:** 20 + 10 = 30
4. **Logarithm base 10 of 100:** log₁₀(100) = 2
5. **Square root of 64:** √64 = 8
6. **Tangent of 45 degrees:** tan(45°) = 1
7. **Cosine of 30 degrees:** cos(30°) = √3/2 ≈ 0.866

You repeated steps 4, 5, 6, and then asked for the cosine of 30 degrees again.  I've listed each calculation only once in this summary.
